In [1]:
library(sf)
library(tmap)
library(tmaptools)
library(dplyr)
library(lubridate)
library(ggplot2)
library(gstat)
library(sp)

# Create map directory if it doesn't exist
if (!dir.exists("./map")) {
  dir.create("./map")
}

# Read the census shapefile
census <- st_read(dsn="./data/nyc2020_census/nyct2020.shp", layer="nyct2020")

# Load the population csv data
population <- read.csv("./data/nyc_censusdata_2020.csv")
population$BCT2020 <- as.character(population$BCT2020)

# Join population data to census data
census_joined <- census %>%
  left_join(population, by= c("BoroCT2020"="BCT2020"))

# Filter out Staten Island
census_joined <- census_joined %>%
  filter(BoroName != "Staten Island")


# Load Citibike data
citibike_1 <- read.csv("./data/2019-citibike-tripdata/7_July/201907-citibike-tripdata_1.csv")
citibike_2 <- read.csv("./data/2019-citibike-tripdata/7_July/201907-citibike-tripdata_2.csv")
citibike_3 <- read.csv("./data/2019-citibike-tripdata/7_July/201907-citibike-tripdata_3.csv")

citibike_july <- bind_rows(citibike_1, citibike_2, citibike_3)

# Convert starttime and stoptime into DateTime format
citibike_july$starttime <- ymd_hms(citibike_july$starttime)
citibike_july$stoptime <- ymd_hms(citibike_july$stoptime)

# Cleaning up for weekday/weekend and hour information
citibike_july <- citibike_july %>%
  mutate(
    # Weekday or weekend
    is_weekend = ifelse(wday(starttime) %in% c(1,7), TRUE, FALSE),
    # Start time hour
    start_hour = hour(starttime)
  )

# Create the station dataset with attributes on station id/lat/long, ride counts, user counts and time-based counts
start_station_july <- citibike_july %>%
  # Group by start.station and user type
  group_by(start.station.id, start.station.latitude, start.station.longitude) %>%
  summarise(
    ride_start_count = n(),
    usertype_subscriber_count = sum(usertype == "Subscriber"),
    usertype_customer_count = sum(usertype == "Customer"),
    
    # Weekday counts for time intervals AM/PM Peak/Off-Peak - ref Liu et al
    ride_start_weekday_7_10 = sum(!is_weekend & start_hour>=7 & start_hour<10),
    ride_start_weekday_10_17 = sum(!is_weekend & start_hour>=10 & start_hour<17),
    ride_start_weekday_17_0 = sum(!is_weekend & start_hour>=17 & start_hour<24),
    ride_start_weekday_0_7 = sum(!is_weekend & start_hour>=0 & start_hour<7),
    
    # Weekend counts for leisure/others - ref Liu et al
    ride_start_weekend_10_0 = sum(is_weekend & start_hour>=10 & start_hour<24),
    ride_start_weekend_0_10 = sum(is_weekend & start_hour>=0 & start_hour<10)
  ) %>%
  rename(
    station_id = start.station.id,
    station_latitude = start.station.latitude,
    station_longitude = start.station.longitude
  )

# Calculate end station counts  
end_station_july <- citibike_july %>%
  group_by(end.station.id, end.station.latitude, end.station.longitude) %>%
  summarise(
    ride_end_count = n()
  ) %>%
  rename(
    station_id = end.station.id,
    station_latitude = end.station.latitude,
    station_longitude = end.station.longitude
  )

# Combine start and end to return ride activity by station
station_july <- full_join(start_station_july, end_station_july,
                          by= c("station_id", "station_latitude", "station_longitude")) %>%
  # Deal with NA values
  mutate(
    ride_start_count = ifelse(is.na(ride_start_count), 0, ride_start_count),
    ride_end_count = ifelse(is.na(ride_end_count), 0, ride_end_count),
    ride_activity = ride_start_count + ride_end_count
  )

# Add median trip duration for start stations
start_station_duration <- citibike_july %>%
  group_by(start.station.id) %>%
  summarise(median_trip_duration_start = median(tripduration, na.rm=TRUE)) %>%
  rename(station_id = start.station.id)

# Add median trip duration for end stations
end_station_duration <- citibike_july %>%
  group_by(end.station.id) %>%
  summarise(median_trip_duration_end = median(tripduration, na.rm=TRUE)) %>%
  rename(station_id = end.station.id)

# Merge median trip duration with station_july
station_july <- station_july %>%
  left_join(start_station_duration, by= "station_id") %>%
  left_join(end_station_duration, by= "station_id")

# Convert cleaned July dataset into sf object
station_july_sf <- station_july %>%
  st_as_sf(coords = c("station_longitude", "station_latitude"), crs=4326, remove=FALSE)

# Convert Census polygon crs4269 to crs4329
census_joined <- st_transform(census_joined, crs=st_crs(station_july_sf))

# Creating an aggregated station dataset at census tract level
station_july_nyc <- station_july_sf %>%
  # Left join to remove stations outside NYC (do not have census tract)
  st_join(census_joined, join= st_within, left=FALSE)
ct_station_july <- station_july_nyc %>%
  group_by(BoroCT2020) %>%
  summarise(
    num_stations = n(),
    total_ride_start_count = sum(ride_start_count, na.rm=TRUE),
    total_ride_end_count = sum(ride_end_count, na.rm=TRUE),
    total_ride_activity = sum(ride_activity, na.rm=TRUE),
    median_trip_duration_start = median(median_trip_duration_start, na.rm=TRUE),
    median_trip_duration_end = median(median_trip_duration_end, na.rm=TRUE),
    usertype_subscriber_count = sum(usertype_subscriber_count, na.rm=TRUE),
    usertype_customer_count = sum(usertype_customer_count, na.rm=TRUE),
    # Adding in the time factor
    weekday_7_10 = sum(ride_start_weekday_7_10, na.rm=TRUE),
    weekday_10_17 = sum(ride_start_weekday_10_17, na.rm=TRUE),
    weekday_17_0 = sum(ride_start_weekday_17_0, na.rm=TRUE),
    weekday_0_7 = sum(ride_start_weekday_0_7, na.rm=TRUE),
    weekend_10_0 = sum(ride_start_weekend_10_0, na.rm=TRUE),
    weekend_0_10 = sum(ride_start_weekend_0_10, na.rm=TRUE)
  )

# Converting to df to allow for inner join and drop column for polygons without Citibike docks
ct_station_july_df <- as.data.frame(ct_station_july)
agg_citibike_july <- census_joined %>%
  inner_join(ct_station_july_df, by= "BoroCT2020")

Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE

Breaking News: tmap 3.x is retiring. Please test v4, e.g. with
remotes::install_github('r-tmap/tmap')


载入程序包：‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



载入程序包：‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




Reading layer `nyct2020' from data source 
  `/Users/jiazhuangfeng/Documents/project/spatial_analysis/assignment/data/nyc2020_census/nyct2020.shp' 
  using driver `ESRI Shapefile'
Simple feature collection with 2325 features and 14 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 913175.1 ymin: 120128.4 xmax: 1067383 ymax: 272844.3
Projected CRS: NAD83 / New York Long Island (ftUS)


`summarise()` has grouped output by 'start.station.id', 'start.station.latitude'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'end.station.id', 'end.station.latitude'. You can override using the `.groups` argument.


In [2]:
ct_station_july_df <- as.data.frame(ct_station_july)
agg_citibike_july <- census_joined %>%
  inner_join(ct_station_july_df, by= "BoroCT2020")

map3 <- tm_shape(census_joined) +
  tm_borders(col= "white") +
  tm_fill(col= "lightgrey") +
  tm_shape(agg_citibike_july) +
  tm_polygons(col= "total_ride_activity",  # 修改此处
              style= "quantile",
              palette= "YlOrRd",
              title= "Total Rides") +  # 修改此处
  tm_layout(title= "Citibike Ride Activity by Census Tract (July 2019)",
            title.position= c("left", "top"),
            legend.position= c("left", "top"),
            legend.outside= FALSE,
            frame= FALSE)

tmap_save(map3, "./map/citibike_ride_activity_census_tract.png")


Map saved to /Users/jiazhuangfeng/Documents/project/spatial_analysis/assignment/map/citibike_ride_activity_census_tract.png

Resolution: 1763.409 by 2500.837 pixels

Size: 5.878031 by 8.336124 inches (300 dpi)



In [3]:
colnames(agg_citibike_july)

[1] "CTLabel"                    "BoroCode"                  
 [3] "BoroName"                   "CT2020"                    
 [5] "BoroCT2020"                 "CDEligibil"                
 [7] "NTAName"                    "NTA2020"                   
 [9] "CDTA2020"                   "CDTANAME"                  
[11] "GEOID"                      "PUMA"                      
[13] "Shape_Leng"                 "Shape_Area"                
[15] "Year"                       "GeoType"                   
[17] "Borough"                    "GeoID"                     
[19] "Pop1"                       "Male.P"                    
[21] "FemP"                       "PopU5"                     
[23] "Pop5t9"                     "Pop10t14"                  
[25] "Pop15t19"                   "Pop20t24"                  
[27] "Pop25t29"                   "Pop30t34"                  
[29] "Pop35t39"                   "Pop40t44"                  
[31] "Pop45t49"                   "Pop50t54"                  
[33] "Pop55t59"                   "Pop60t64"                  
[35] "Pop65t69"                   "Pop70t74"                  
[37] "Pop75t79"                   "Pop80t84"                  
[39] "Pop85pl"                    "MdAge"                     
[41] "PopU18"                     "Pop65pl"                   
[43] "GQClgHsg"                   "Fam"                       
[45] "HUnits"                     "OcHU_1P"                   
[47] "AvgHHSz"                    "num_stations"              
[49] "total_ride_start_count"     "total_ride_end_count"      
[51] "total_ride_activity"        "median_trip_duration_start"
[53] "median_trip_duration_end"   "usertype_subscriber_count" 
[55] "usertype_customer_count"    "weekday_7_10"              
[57] "weekday_10_17"              "weekday_17_0"              
[59] "weekday_0_7"                "weekend_10_0"              
[61] "weekend_0_10"               "geometry.x"                
[63] "geometry.y"

In [4]:
ct_station_july_df <- as.data.frame(ct_station_july)
agg_citibike_july <- census_joined %>%
  inner_join(ct_station_july_df, by= "BoroCT2020")

num_columns <- ncol(agg_citibike_july)
cat("列的数量:", num_columns, "\n")
filtered_data <- agg_citibike_july %>%
  filter(total_ride_activity > 0)
num_columns <- ncol(filtered_data)
cat("列的数量:", num_columns, "\n")


列的数量: 63 
列的数量: 63 


In [5]:
colnames(agg_citibike_july)
head(agg_citibike_july$total_ride_activity)

[1] "CTLabel"                    "BoroCode"                  
 [3] "BoroName"                   "CT2020"                    
 [5] "BoroCT2020"                 "CDEligibil"                
 [7] "NTAName"                    "NTA2020"                   
 [9] "CDTA2020"                   "CDTANAME"                  
[11] "GEOID"                      "PUMA"                      
[13] "Shape_Leng"                 "Shape_Area"                
[15] "Year"                       "GeoType"                   
[17] "Borough"                    "GeoID"                     
[19] "Pop1"                       "Male.P"                    
[21] "FemP"                       "PopU5"                     
[23] "Pop5t9"                     "Pop10t14"                  
[25] "Pop15t19"                   "Pop20t24"                  
[27] "Pop25t29"                   "Pop30t34"                  
[29] "Pop35t39"                   "Pop40t44"                  
[31] "Pop45t49"                   "Pop50t54"                  
[33] "Pop55t59"                   "Pop60t64"                  
[35] "Pop65t69"                   "Pop70t74"                  
[37] "Pop75t79"                   "Pop80t84"                  
[39] "Pop85pl"                    "MdAge"                     
[41] "PopU18"                     "Pop65pl"                   
[43] "GQClgHsg"                   "Fam"                       
[45] "HUnits"                     "OcHU_1P"                   
[47] "AvgHHSz"                    "num_stations"              
[49] "total_ride_start_count"     "total_ride_end_count"      
[51] "total_ride_activity"        "median_trip_duration_start"
[53] "median_trip_duration_end"   "usertype_subscriber_count" 
[55] "usertype_customer_count"    "weekday_7_10"              
[57] "weekday_10_17"              "weekday_17_0"              
[59] "weekday_0_7"                "weekend_10_0"              
[61] "weekend_0_10"               "geometry.x"                
[63] "geometry.y"

[1] 24762 11502 41664  6513  5930  9663

In [6]:
library(sf)
library(spdep)

# 确保数据是 sf 对象
# agg_citibike_july <- st_as_sf(agg_citibike_july)

# 创建邻接矩阵
nb <- poly2nb(agg_citibike_july, row.names = agg_citibike_july$GEOID)

# 将邻接矩阵转换为权重矩阵
listw <- nb2listw(nb, style = "W", zero.policy = TRUE)

# 查看权重矩阵
print(listw)


载入需要的程序包：spData

To access larger datasets in this package, install the spDataLarge
package with: `install.packages('spDataLarge',
repos='https://nowosad.github.io/drat/', type='source')`

Warning message in poly2nb(agg_citibike_july, row.names = agg_citibike_july$GEOID):
“some observations have no neighbours;
if this seems unexpected, try increasing the snap argument.”
Warning message in poly2nb(agg_citibike_july, row.names = agg_citibike_july$GEOID):
“neighbour object has 3 sub-graphs;
if this sub-graph count seems unexpected, try increasing the snap argument.”


Characteristics of weights list object:
Neighbour list object:
Number of regions: 441 
Number of nonzero links: 2258 
Percentage nonzero weights: 1.161039 
Average number of links: 5.120181 
1 region with no links:
36047001802
3 disjoint connected subgraphs

Weights style: W 
Weights constants summary:
    n     nn  S0       S1       S2
W 440 193600 440 189.1581 1817.575


In [7]:
library(spdep)

# 假设 nb 和 listw 已经创建
# nb <- poly2nb(agg_citibike_july, row.names = agg_citibike_july$GEOID, snap = 0.01)
# listw <- nb2listw(nb, style = "W", zero.policy = TRUE)

# 提取 total_ride_activity 作为变量
zj <- agg_citibike_july$total_ride_activity

# 计算 Moran's I
moran_test <- moran.test(zj, listw, zero.policy = TRUE)

# 输出结果
print(moran_test)



	Moran I test under randomisation

data:  zj  
weights: listw  
n reduced by no-neighbour observations  

Moran I statistic standard deviate = 14.787, p-value < 2.2e-16
alternative hypothesis: greater
sample estimates:
Moran I statistic       Expectation          Variance 
     0.4460428733     -0.0022779043      0.0009192388 



In [54]:
library(ggplot2)
library(sf)
library(spdep)

# 提取 total_ride_activity 作为变量
zj <- agg_citibike_july$total_ride_activity

# 计算局部 Moran's I
local_moran <- localmoran(zj, listw, zero.policy = TRUE)

# 将结果添加到数据框
agg_citibike_july$local_moran_I <- local_moran[,1]
agg_citibike_july$local_moran_p_value <- local_moran[,5]

# 使用 ggplot2 可视化
p <- ggplot() +
  geom_sf(data = census_joined, fill = "grey90", color = "white", size = 0.2) + # 背景地图
  geom_sf(data = agg_citibike_july, aes(fill = local_moran_I), color = "black", size = 0.2) +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0, 
                       name = "Local Moran's I", 
                       na.value = "grey80") +
  theme_minimal() +
  labs(title = "Local Moran's I for Citibike Activity",
       subtitle = "Spatial Autocorrelation of Ride Activity") +
  theme(legend.position = "bottom")

# 保存图形
ggsave(filename = "./map/local_morans_I_citibike.png", plot = p, width = 10, height = 8, dpi = 300)


In [56]:
library(dplyr)
library(ggplot2)
library(sf)

# 添加显著性标记
agg_citibike_july <- agg_citibike_july %>%
  mutate(significance = case_when(
    local_moran_p_value < 0.001 ~ "99.9% level",
    local_moran_p_value < 0.01 ~ "99% level",
    local_moran_p_value < 0.05 ~ "95% level",
    TRUE ~ "Not significant"
  ))

# 绘制地图
p <- ggplot() +
  geom_sf(data = census_joined, fill = "grey90", color = "white", size = 0.2) + # 背景地图
  geom_sf(data = agg_citibike_july, aes(fill = significance), color = "black", size = 0.2) +
  scale_fill_manual(values = c("99.9% level" = "red", "99% level" = "orange", "95% level" = "yellow", "Not significant" = "grey80"),
                    name = "Significance Level") +
  theme_minimal() +
  labs(title = "Significance of Local Moran's I for Citibike Activity",
       subtitle = "Spatial Autocorrelation of Ride Activity") +
  theme(legend.position = "bottom")

# 保存图形
ggsave(filename = "./map/local_morans_I_significance.png", plot = p, width = 10, height = 8, dpi = 300)


In [58]:
# 添加聚集类型标记
agg_citibike_july <- agg_citibike_july %>%
  mutate(cluster_type = case_when(
    local_moran_p_value < 0.01 & local_moran_I > 0 & total_ride_activity > mean(total_ride_activity) ~ "High-High",
    local_moran_p_value < 0.01 & local_moran_I > 0 & total_ride_activity < mean(total_ride_activity) ~ "Low-Low",
    TRUE ~ "Not significant"
  ))

# 绘制地图
p <- ggplot() +
  geom_sf(data = census_joined, fill = "grey90", color = "white", size = 0.2) + # 背景地图
  geom_sf(data = agg_citibike_july, aes(fill = cluster_type), color = "black", size = 0.2) +
  scale_fill_manual(values = c("High-High" = "yellow", "Low-Low" = "red", "Not significant" = "grey80"),
                    name = "Cluster Type") +
  theme_minimal() +
  labs(title = "Cluster Types of Local Moran's I for Citibike Activity",
       subtitle = "Spatial Clustering of Ride Activity at 99% Confidence Interval") +
  theme(legend.position = "bottom")

# 保存图形
ggsave(filename = "./map/local_morans_I_cluster_types.png", plot = p, width = 10, height = 8, dpi = 300)

In [59]:
# 查看数据分布
summary(agg_citibike_july$total_ride_activity)

# 检查显著性和聚集类型
table(agg_citibike_july$cluster_type)

# 查看局部 Moran's I 和 p 值
head(agg_citibike_july[, c("local_moran_I", "local_moran_p_value", "total_ride_activity")])


   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
      1    2240    5417    9891   12189  131506 


      High-High Not significant 
             60             381 

,local_moran_I,local_moran_p_value,total_ride_activity,geometry.x
,<dbl>,<dbl>,<dbl>,<MULTIPOLYGON [°]>
1,0.69307292,0.302524734,24762,MULTIPOLYGON (((-73.99022 4...
2,0.15091561,0.007660137,11502,MULTIPOLYGON (((-73.98837 4...
3,2.17931951,0.030020959,41664,MULTIPOLYGON (((-73.98985 4...
4,-0.05218360,0.630858797,6513,MULTIPOLYGON (((-73.97875 4...
5,-0.21869483,0.116579713,5930,MULTIPOLYGON (((-73.97689 4...
6,-0.02624991,0.003488313,9663,MULTIPOLYGON (((-73.9733 40...
